# **Agentic Sales Pipeline SambaNova & CrewAI**

This tutorial and companion notebook demonstrate how to create an **Agentic Sales Pipeline** using **CrewAI** integrated with **SambaNova's Meta-Llama 3.1 70B Instruct** model. The workflow automates lead qualification, scoring, and personalized email engagement by orchestrating agents that collect data, analyze cultural fit, and draft emails. The pipeline showcases modular design, high inference performance, and the ability to visualize and execute multi-step AI workflows efficiently.


## **Introduction**

In the world of sales, time is money. Manually qualifying leads, scoring them, and drafting follow-up emails can be tedious and inefficient. **Agentic AI workflows** powered by **CrewAI** and **SambaNova’s LLMs** can automate these tasks. This guide will walk you through setting up an **Agentic Sales Pipeline** to qualify, score, and engage leads.

We’ll break down each step, explaining how **CrewAI’s components**—**Agents**, **Tasks**, **Crews**, and **Flows**—work together to accomplish these goals.

## **Understanding the Core Components of CrewAI**

Before we dive into the code, let’s explore the key building blocks of CrewAI:

**The LLM Class:** Encapsulates a **Large Language Model** used by agents to perform tasks. In this pipeline, we use **SambaNova’s Meta-Llama 3.1 70B Instruct** model, known for its high speed and power efficiency.

**The Task Class:** Defines **specific objectives** for agents. Each task contains a Goal (what the agent should accomplish), an Agent (the AI worker assigned to the task) and Tools (optional utilities, like a web search, to assist the agent). Tasks are modular and can be easily modified or reused.

**The Crew Class:** Brings agents together to work collaboratively, managing task execution, communication between agents, and providing debugging information.

With these components in mind, let’s set up our environment.

## **1\. Setting Up the Environment** 

First, make sure to install the needed dependencies from the repo.

In [ ]:
# Install Requirements

!pip install -r requirements.txt

In [2]:
import os
import yaml

import asyncio
import getpass
import nest_asyncio
import textwrap
import pandas as pd
import warnings

from crewai import  Agent, Crew, Flow, LLM, Task
from crewai.flow.flow import listen, start, and_, or_, router
from crewai_tools import SerperDevTool, ScrapeWebsiteTool
from dotenv import load_dotenv
from litellm import cost_per_token
from pydantic import BaseModel, Field
from typing import Optional, List

from IPython.display import display, HTML, IFrame

warnings.filterwarnings('ignore')
nest_asyncio.apply()

Second, we’ll need to install dependencies and configure API keys. We need the following API keys:

Get your Sambanova API Key Here https://cloud.sambanova.ai/

Get your Serper API Key Here https://serper.dev/ 

In [3]:
# Set API keys and base URL if not already configured in environment

load_dotenv()

if "SAMBANOVA_API_KEY" not in os.environ:
    os.environ["SAMBANOVA_API_KEY"] = getpass.getpass("insert your SambaNova API Key") 
    
if "SAMBANOVA_BASE_URL" not in os.environ:
    os.environ["SAMBANOVA_BASE_URL"] = "https://api.sambanova.ai/v1"
    
if "SERPER_API_KEY" not in os.environ:
    os.environ["SERPER_API_KEY"] = getpass.getpass("insert your Serper API Key") 



## **2\. Initializing the LLM** 

To make LLM calls, we’ll need to set up the SambaNova LLM for agents to use in their tasks. We’ll initialize the **Meta-Llama 3.1 70B Instruct** model, which agents will use to process and generate text.

In [4]:
# Set Up Sambanova LLM
model = 'sambanova/Meta-Llama-3.1-70B-Instruct'
llm = LLM(
    model=model, 
    base_url=os.getenv("SAMBANOVA_BASE_URL"),
    api_key=os.getenv("SAMBANOVA_API_KEY")
    )

## **3\. Loading Agent and Task Configurations** 

Next, we’ll need to define some agents and tasks to deliver our use case. CrewAI promotes modularity by allowing you to define agents and tasks in YAML files. This makes it easy to update roles and objectives without changing the code. 


In [5]:
# Define file paths for YAML configurations
files = {
    'lead_agents': 'config/lead_qualification_agents.yaml',
    'lead_tasks': 'config/lead_qualification_tasks.yaml',
    'email_agents': 'config/email_engagement_agents.yaml',
    'email_tasks': 'config/email_engagement_tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
lead_agents_config = configs['lead_agents']
lead_tasks_config = configs['lead_tasks']
email_agents_config = configs['email_agents']
email_tasks_config = configs['email_tasks']

### Create Pydantic Models for Structured Output

In [6]:
class LeadPersonalInfo(BaseModel):
    name: str = Field(..., description="The full name of the lead.")
    job_title: str = Field(..., description="The job title of the lead.")
    role_relevance: int = Field(..., ge=0, le=10, description="A score representing how relevant the lead's role is to the decision-making process (0-10).")
    professional_background: Optional[str] = Field(..., description="A brief description of the lead's professional background.")

class CompanyInfo(BaseModel):
    company_name: str = Field(..., description="The name of the company the lead works for.")
    industry: str = Field(..., description="The industry in which the company operates.")
    company_size: int = Field(..., description="The size of the company in terms of employee count.")
    revenue: Optional[float] = Field(None, description="The annual revenue of the company, if available.")
    market_presence: int = Field(..., ge=0, le=10, description="A score representing the company's market presence (0-10).")

class LeadScore(BaseModel):
    score: int = Field(..., ge=0, le=100, description="The final score assigned to the lead (0-100).")
    scoring_criteria: List[str] = Field(..., description="The criteria used to determine the lead's score.")
    validation_notes: Optional[str] = Field(None, description="Any notes regarding the validation of the lead score.")

class LeadScoringResult(BaseModel):
    personal_info: LeadPersonalInfo = Field(..., description="Personal information about the lead.")
    company_info: CompanyInfo = Field(..., description="Information about the lead's company.")
    lead_score: LeadScore = Field(..., description="The calculated score and related information for the lead.")

## **4\. Creating Agents** 

Task: Define agents responsible for lead qualification and scoring.

Agents are the AI workers that perform tasks. We’ll define three agents:

1. **Lead Data Agent**: Collects lead information using search and scraping tools.  
2. **Cultural Fit Agent**: Assesses if the lead aligns with company values.  
3. **Scoring Validation Agent**: Validates and scores the lead.

For each agent, we’ll also define:

* **Tools**: Agents use **Serper** for search and **ScrapeWebsiteTool** for web scraping.  
* **LLM**: Each agent uses the **Meta-Llama 3.1** model to process and generate responses.

In [7]:
# Creating Agents
lead_data_agent = Agent(
  config=lead_agents_config['lead_data_agent'],
  tools=[SerperDevTool(), ScrapeWebsiteTool()],
  llm=llm
)

cultural_fit_agent = Agent(
  config=lead_agents_config['cultural_fit_agent'],
  tools=[SerperDevTool(), ScrapeWebsiteTool()],
  llm=llm
)

scoring_validation_agent = Agent(
  config=lead_agents_config['scoring_validation_agent'],
  tools=[SerperDevTool(), ScrapeWebsiteTool()],
  llm=llm
)



## **5\. Defining Tasks** 

Next, we’ll need to create the tasks that agents will execute. Each task specifies what the agent needs to accomplish. For example:

* **Lead Data Collection**: Collects basic information about leads.  
* **Cultural Fit Analysis**: Analyzes lead fit.  
* **Lead Scoring**: Scores and validates leads.

In [8]:
# Creating Tasks
lead_data_task = Task(
  config=lead_tasks_config['lead_data_collection'],
  agent=lead_data_agent
)

cultural_fit_task = Task(
  config=lead_tasks_config['cultural_fit_analysis'],
  agent=cultural_fit_agent
)

scoring_validation_task = Task(
  config=lead_tasks_config['lead_scoring_and_validation'],
  agent=scoring_validation_agent,
  context=[lead_data_task, cultural_fit_task],
  output_pydantic=LeadScoringResult
)

## **6\. Creating the Crew** 

Next, we’ll combine agents and tasks into a **Crew** to orchestrate the workflow. The **Crew** ensures tasks are executed in order and agents collaborate seamlessly, while verbose logging helps track progress and debug issues.

In [9]:
# Creating Crew
lead_scoring_crew = Crew(
  agents=[
    lead_data_agent,
    cultural_fit_agent,
    scoring_validation_agent
  ],
  tasks=[
    lead_data_task,
    cultural_fit_task,
    scoring_validation_task
  ],
  verbose=True
)

### Repeat Steps 4-6 for Email Engagement Crew

In [ ]:
# Creating Agents
email_content_specialist = Agent(
  config=email_agents_config['email_content_specialist'],
  llm=llm
)

engagement_strategist = Agent(
  config=email_agents_config['engagement_strategist'],
  llm=llm
)

# Creating Tasks
email_drafting = Task(
  config=email_tasks_config['email_drafting'],
  agent=email_content_specialist
)

engagement_optimization = Task(
  config=email_tasks_config['engagement_optimization'],
  agent=engagement_strategist
)

# Creating Crew
email_writing_crew = Crew(
  agents=[
    email_content_specialist,
    engagement_strategist
  ],
  tasks=[
    email_drafting,
    engagement_optimization
  ],
  verbose=True,
)

## **7\. Creating the Sales Pipeline Flow** 

Task: Automate the sales pipeline process using a CrewAI Flow.

In this step, we'll define a **Flow** that orchestrates the end-to-end sales process. The Flow will:

1. **Fetch Leads**: Retrieve potential leads (mock data).  
2. **Score Leads**: Run the lead scoring Crew to qualify and score the leads.  
3. **Store Scores**: Save the lead scores (simulated).  
4. **Filter High-Quality Leads**: Identify leads with scores above a threshold.  
5. **Draft and Send Emails**: Generate personalized emails for high-quality leads.

In [11]:
class SalesPipeline(Flow):
    @start()
    def fetch_leads(self):
        # Pull our leads from the database
        leads = [
            {
                "lead_data": {
                    "name": "Kwasi Ankomah",
                    "job_title": "Architect",
                    "company": "SambaNova",
                    "email": "kwasi@samaba.com",
                    "use_case": "Using AI Agents to do better data enrichment."
                },
            },
        ]
        return leads

    @listen(fetch_leads)
    def score_leads(self, leads):
        scores = lead_scoring_crew.kickoff_for_each(leads)
        self.state["score_crews_results"] = scores
        return scores

    @listen(score_leads)
    def store_leads_score(self, scores):
        # Here we would store the scores in the database
        return scores

    @listen(score_leads)
    def filter_leads(self, scores):
        return [score for score in scores if score['lead_score'].score > 70]

    @listen(filter_leads)
    def write_email(self, leads):
        scored_leads = [lead.to_dict() for lead in leads]
        emails = email_writing_crew.kickoff_for_each(scored_leads)
        return emails

    @listen(write_email)
    def send_email(self, emails):
        # Here we would send the emails to the leads
        return emails

flow = SalesPipeline()

## **8\. Visualizing the Flow** 

CrewAI allows you to generate a visual representation of your Flow, which is useful for debugging and documentation. This will generate a diagram showing the sequence of steps in the sales pipeline from lead fetching to sending emails (simulated).
* Note that Jupyter Notebooks is the preferred system for running and viewing the visualizations. 

In [ ]:
flow.plot()

In [ ]:
IFrame(src='./crewai_flow.html', width='150%', height=600)

![crew flow](crewai_flow.png "CrewAI Flow")

## **9\. Executing the Flow** 

If all looks good, we’re ready to run the Flow to process leads through the entire pipeline. We use `nest_asyncio` to ensure the async execution works smoothly in a notebook environment.

In [ ]:
async def run_flow():
    return flow.kickoff()

emails = asyncio.run(run_flow())

### Results

In [ ]:
result_text = emails[0].raw
wrapped_text = textwrap.fill(result_text, width=80)
print(wrapped_text)

### Usage Metrics and Costs

Let’s see how much it would cost each time if this crew runs at scale.

In [ ]:
# Lead qualification crew
# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([flow.state["score_crews_results"][0].token_usage.dict()])

# Calculate total costs
prompt_tokens = df_usage_metrics['prompt_tokens'].sum() 
completion_tokens = df_usage_metrics['completion_tokens'].sum()

prompt_tokens_cost_usd_dollar, completion_tokens_cost_usd_dollar = cost_per_token(model=model, prompt_tokens=prompt_tokens, completion_tokens=completion_tokens)

costs = prompt_tokens_cost_usd_dollar + completion_tokens_cost_usd_dollar 
print(f"Total costs: ${costs:.4f}")

# Display the DataFrame
df_usage_metrics

In [ ]:
# Email engagement crew
# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([emails[0].token_usage.dict()])

# Calculate total costs
prompt_tokens = df_usage_metrics['prompt_tokens'].sum() 
completion_tokens = df_usage_metrics['completion_tokens'].sum()

prompt_tokens_cost_usd_dollar, completion_tokens_cost_usd_dollar = cost_per_token(model=model, prompt_tokens=prompt_tokens, completion_tokens=completion_tokens)

costs = prompt_tokens_cost_usd_dollar + completion_tokens_cost_usd_dollar 
print(f"Total costs: ${costs:.4f}")

# Display the DataFrame
df_usage_metrics

## **10\. Inspecting the Results** 

Now, let’s examine the output to see how the leads were scored and what emails were generated.

In [18]:
scores = flow.state["score_crews_results"]

### View Lead Scores

In [ ]:
lead_scoring_result = scores[0].pydantic

# Create a dictionary with the nested structure flattened
data = {
    'Name': lead_scoring_result.personal_info.name,
    'Job Title': lead_scoring_result.personal_info.job_title,
    'Role Relevance': lead_scoring_result.personal_info.role_relevance,
    'Professional Background': lead_scoring_result.personal_info.professional_background,
    'Company Name': lead_scoring_result.company_info.company_name,
    'Industry': lead_scoring_result.company_info.industry,
    'Company Size': lead_scoring_result.company_info.company_size,
    'Revenue': lead_scoring_result.company_info.revenue,
    'Market Presence': lead_scoring_result.company_info.market_presence,
    'Lead Score': lead_scoring_result.lead_score.score,
    'Scoring Criteria': ', '.join(lead_scoring_result.lead_score.scoring_criteria),
    'Validation Notes': lead_scoring_result.lead_score.validation_notes
}

# Convert the dictionary to a DataFrame
df = pd.DataFrame.from_dict(data, orient='index', columns=['Value'])

# Reset the index to turn the original column names into a regular column
df = df.reset_index()

# Rename the index column to 'Attribute'
df = df.rename(columns={'index': 'Attribute'})

# Create HTML table with bold attributes and left-aligned values
html_table = df.style.set_properties(**{'text-align': 'left'}) \
                     .format({'Attribute': lambda x: f'<b>{x}</b>'}) \
                     .hide(axis='index') \
                     .to_html()

# Display the styled HTML table
display(HTML(html_table))

### **How Complex Can it Get?**

In [20]:
class SalesPipeline(Flow):
    
  @start()
  def fetch_leads(self):
    # Pull our leads from the database
    # This is a mock, in a real-world scenario, this is where you would
    # fetch leads from a database
    leads = [
      {
        "lead_data": {
          "name": "Kwasi Ankomah",
          "job_title": "Architect",
          "company": "Sambanova",
          "email": "kwasi@sambanova.com",
          "use_case": "Using AI Agent to do better data enrichment."
        },
      },
    ]
    return leads

  @listen(fetch_leads)
  def score_leads(self, leads):
    scores = lead_scoring_crew.kickoff_for_each(leads)
    self.state["score_crews_results"] = scores
    return scores

  @listen(score_leads)
  def store_leads_score(self, scores):
    # Here we would store the scores in the database
    return scores

  @listen(score_leads)
  def filter_leads(self, scores):
    return [score for score in scores if score['lead_score'].score > 70]

  @listen(and_(filter_leads, store_leads_score))
  def log_leads(self, leads):
    print(f"Leads: {leads}")

  @router(filter_leads)
  def count_leads(self, scores):
    if len(scores) > 10:
      return 'high'
    elif len(scores) > 5:
      return 'medium'
    else:
      return 'low'

  @listen('high')
  def store_in_salesforce(self, leads):
    return leads

  @listen('medium')
  def send_to_sales_team(self, leads):
    return leads

  @listen('low')
  def write_email(self, leads):
    scored_leads = [lead.to_dict() for lead in leads]
    emails = email_writing_crew.kickoff_for_each(scored_leads)
    return emails

  @listen(write_email)
  def send_email(self, emails):
    # Here we would send the emails to the leads
    return emails

### Plotting the Flow

In [ ]:
flow = SalesPipeline()
flow.plot()

In [ ]:
IFrame(src='./crewai_flow.html', width='150%', height=600)

## **Final Thoughts**

In this notebook, we demonstrated how to build and orchestrate a multi-agent workflow using **CrewAI** and **SambaNova’s LLMs**. We accomplished key technical milestones, including:

* **Modular Agent Design**: Defined agents and tasks using flexible YAML configurations, making it easy to adapt the workflow to different domains or use cases.  
* **LLM Integration**: Leveraged **SambaNova's Meta-Llama 3.1 70B Instruct** model to power agents with high-speed inference and low-latency responses.  
* **Tool Integration**: Combined agents with tools like web search and scraping to enrich tasks with real-time data.  
* **Orchestrated Automation**: Used the **Crew** and **Flow** classes to create seamless, multi-step workflows with clear task delegation and output management.  
* **Structured Outputs**: Implemented **Pydantic models** to ensure consistent and validated data outputs across tasks.  
* **Workflow Visualization**: Generated visual representations of the flow to understand task dependencies and execution paths.  
* **Cost Estimation**: Measured token usage and estimated costs to help gauge the efficiency of the workflow.

These technical capabilities provide a blueprint for building scalable and customizable agentic workflows. Whether you’re automating data enrichment, research tasks, customer support, or content generation, you can easily define and modify agents, incorporate real-time tools, debug flows, and monitor performance and costs.

By applying these principles, developers can automate sophisticated, multi-step processes, making their workflows more intelligent, adaptable, and efficient.

### **Ready to Automate Your Own Use Case?**

Start building with **SambaNova** and **CrewAI** today\! For more information, check out the [SambaNova Cloud](https://cloud.sambanova.ai) Documentation.
